# Magentic Orchestration Example

This notebook demonstrates a complete Magentic workflow using Microsoft Agent Framework. The workflow coordinates two specialist agents:
- **Researcher**: Finds information using web search
- **Coder**: Executes Python code for analysis

The example shows streaming callbacks, event handling, and complete workflow execution.

In [ ]:
import os

from agent_framework import (
    ChatAgent,
    HostedCodeInterpreterTool,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticBuilder,
    MagenticCallbackEvent,
    MagenticCallbackMode,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
    WorkflowOutputEvent,
)
from agent_framework.openai import OpenAIChatClient, OpenAIResponsesClient

# Set OpenAI API key (ensure OPENAI_API_KEY is set in environment)
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

print("✓ Imports loaded successfully")

## Step 1: Configure Researcher Agent

The ResearcherAgent specializes in finding information. It uses the `gpt-4o-search-preview` model to perform web searches.


In [ ]:
researcher_agent = ChatAgent(
    name="ResearcherAgent",
    description="Specialist in research and information gathering",
    instructions=(
        "You are a Researcher. You find information without additional computation or quantitative analysis."
    ),
    chat_client=OpenAIChatClient(model_id="gpt-4o-search-preview"),
)


print("✓ Researcher agent configured")


## Step 2: Configure Coder Agent

The CoderAgent writes and executes code using the hosted code interpreter tool.


In [ ]:
coder_agent = ChatAgent(
    name="CoderAgent",
    description="A helpful assistant that writes and executes code to process and analyze data.",
    instructions="You solve questions using code. Please provide detailed analysis and computation process.",
    chat_client=OpenAIResponsesClient(model_id="gpt-4o"),
    tools=[HostedCodeInterpreterTool()],
)

print("✓ Coder agent configured")

## Step 3: Define Event Callback

The callback processes workflow events including orchestrator messages, streaming deltas, agent messages, and final results.


In [ ]:
# State for tracking streaming output

last_stream_agent_id: str | None = None

stream_line_open: bool = False


async def on_event(event: MagenticCallbackEvent) -> None:
    """Process workflow events such as orchestrator messages, streaming deltas, agent messages, and final results.

    """
    global last_stream_agent_id, stream_line_open

    if isinstance(event, MagenticOrchestratorMessageEvent):
        print(f"\n[ORCH:{event.kind}]\n\n{getattr(event.message, 'text', '')}\n{'-' * 26}")

    elif isinstance(event, MagenticAgentDeltaEvent):
        if last_stream_agent_id != event.agent_id or not stream_line_open:
            if stream_line_open:
                print()

            print(f"\n[STREAM:{event.agent_id}]: ", end="", flush=True)

            last_stream_agent_id = event.agent_id

            stream_line_open = True

        print(event.text, end="", flush=True)

    elif isinstance(event, MagenticAgentMessageEvent):
        if stream_line_open:
            print(" (final)")

            stream_line_open = False

            print()

        msg = event.message

        if msg is not None:
            response_text = (msg.text or "").replace("\n", " ")

            print(f"\n[AGENT:{event.agent_id}] {msg.role.value}\n\n{response_text}\n{'-' * 26}")

    elif isinstance(event, MagenticFinalResultEvent):
        print("\n" + "=" * 50)

        print("FINAL RESULT:")

        print("=" * 50)

        if event.message is not None:
            print(event.message.text)

        print("=" * 50)


print("✓ Event callback defined")


## Step 4: Build Workflow

Create the Magentic workflow with both agents and streaming callback support.


In [ ]:
print("Building Magentic workflow...")

workflow = (
    MagenticBuilder()
    .participants(researcher=researcher_agent, coder=coder_agent)
    .on_event(on_event, mode=MagenticCallbackMode.STREAMING)
    .with_standard_manager(
        chat_client=OpenAIResponsesClient(model_id="gpt-4o"),
        max_round_count=10,
        max_stall_count=3,
        max_reset_count=2,
    )
    .build()
)

print("✓ Workflow built successfully")

## Step 5: Define Task

Specify the research task comparing energy efficiency and CO2 emissions of different ML model architectures.


In [ ]:
task = (
    "I am preparing a report on the energy efficiency of different machine learning model architectures. "
    "Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 "
    "on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). "
    "Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 "
    "VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model "
    "per task type (image classification, text classification, and text generation)."
)


print(f"✓ Task defined: {task[:100]}...")


## Step 6: Execute Workflow

Run the workflow and stream the results. This requires valid OpenAI credentials.


In [ ]:
async def run_workflow():
    print("\n" + "=" * 50)

    print("Starting workflow execution...")

    print("=" * 50 + "\n")

    try:
        output: str | None = None

        async for event in workflow.run_stream(task):
            # Events are processed by the on_event callback

            if isinstance(event, WorkflowOutputEvent):
                output = str(event.data)

        if output is not None:
            print(f"\n\nWorkflow completed with result:\n\n{output}")

        else:
            print("\n\nWorkflow completed (no output)")

    except Exception as e:
        print(f"\n\nWorkflow execution failed: {e}")

        raise


# Execute the workflow

await run_workflow()
